In [ ]:
import cv2
import numpy as np
def EasterBunnyFace(model):
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
    capture=cv2.VideoCapture(0)
    print("\n\nEnter 'q' to exit\n\n")
    while True:
        ret,frame=capture.read()
        gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        
        faces= face_cascade.detectMultiScale(gray_frame,1.3,5) #(cascade,scale_factor,minnimum neighbours)
        if(len(faces)==0):
            continue
        
        for (x,y,w,h) in faces:
            copy1_frame = np.copy(frame)
            copy2_frame = np.copy(frame)
            img_det_gray = gray_frame[y:y+h, x:x+w]
            img_det_color = copy1_frame[y:y+h , x:x+h]
            width_final = img_det_gray.shape[1]
            height_final = img_det_gray.shape[0]
            img_gray = cv2.resize(img_det_gray,(96,96))
            img_gray = img_gray/255
            
            image_input_model = np.reshape(img_gray,(1,96,96,1)) #bactchsize,height,width,no. of channels
            facial_keypoints = model.predict(image_input_model)[0]
            
            x_val = facial_keypoints[0::2]
            y_val = facial_keypoints[1::2]
            
            x_coordinates = (x_val+0.5)*width_final
            y_coordinates = (y_val+0.5)*height_final
            
            
            for i in range(len(x_val)):
                cv2.circle(img_det_color,(x_coordinates[i],y_coordinates[i]),2,(255,255,0),-1)
                
            
            left_eye_left_coordinate=(int(x_coordinates[3]),int(y_coordinates[3]))
            right_eye_right_coordinate=(int(x_coordinates[5]),int(y_coordinates[5]))
            brow_coordinate=(int(x_coordinates[6]),int(y_coordinates[6]))
            left_lip_coordinate=(int(x_coordinates[11]),int(y_coordinates[11]))
            right_lip_coordinate=(int(x_coordinates[12]),int(y_coordinates[12]))
            top_lip_coordinate=(int(x_coordinates[13]),int(y_coordinates[13]))
            bottom_lip_coordinate=(int(x_coordinates[14]),int(y_coordinates[14]))
            #alpha channel represents the  degree of transparency 
            copy2_frame = cv2.cvtColor(copy2_frame,cv2.COLOR_BGR2BGRA)
                
                
            bunny_filter = cv2.imread('Filters/Bunny.png',-1)
            #scaling of bunny filter
            bunny_filter = cv2.resize(bunny_filter,(w*2,w*2)) # the width of face
            hx,hy,hz = bunny_filter.shape
                
            #positioning of bunny filter
            for i in range(0,hx):
                for j in range(0,hy):
                    if bunny_filter[i,j][3]!=0:
                        copy2_frame[i + y - brow_coordinate[1]*2 -50,j+x-left_eye_left_coordinate[0] -39] = bunny_filter[i,j]
                                
            copy2_frame=cv2.cvtColor(copy2_frame,cv2.COLOR_BGRA2BGR)
                
            cv2.imshow('Face with Filter',copy2_frame)
            cv2.imshow('Face with Keypoints',copy1_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    capture.release()
    cv2.destroyAllWindows()